In [ ]:
import random
import csv

def luck_money(total, num):
    tmp = num
    for i in range(num-1):       
        hb = round(random.uniform(0.01, total/num*2), 2)
        total = total - hb
        if total <= num*0.01:   
            total = total + hb
            num -= 1
            hb = total - num*0.01
            yield round(hb, 2)
            for j in range(i+1, tmp):   
                yield 0.01
            break                       
        yield hb
        num -= 1
    else:                       
        total = round(total, 2)
        yield total

def generate_lucky_money(total, num, rounds):
    results = []
    for _ in range(rounds):
        generator = luck_money(total, num)
        results.append([next(generator) for _ in range(num)])
    return results


for num_red_packets in range(3, 26):

    total_amount = 200
    rounds = 100000

    red_packet_amounts = generate_lucky_money(total_amount, num_red_packets, rounds)
    filename = f'lucky_money_{num_red_packets}_packets.csv'

    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        headers = ['Round'] + [f'Red Packet {i+1}' for i in range(num_red_packets)]
        writer.writerow(headers)
        
        for round_index, round_amounts in enumerate(red_packet_amounts, start=1):
            writer.writerow([round_index] + round_amounts)

    print(f'CSV file {filename} has been created with {rounds} rounds of lucky money distribution.')


In [ ]:
import csv
import matplotlib.pyplot as plt

# 计算每个位次抢到最大红包的概率
def calculate_max_red_packet_probability(file_path, total_rounds, num_red_packets):
    max_counts = [0] * num_red_packets  
    
    with open(file_path, 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        next(reader, None) 
        
        round_count = 0  
        for row in reader:
            round_count += 1
            amounts = list(map(float, row[1:]))  
            max_amount = max(amounts)
            max_index = amounts.index(max_amount) + 1  
            max_counts[max_index - 1] += 1  
    
    probabilities = [count / total_rounds for count in max_counts]  
    return probabilities, round_count


num_red_packets_range = range(3, 26)
total_rounds = 100000  

# 循环遍历红包数量
for num_red_packets in num_red_packets_range:
    
    file_path = f'lucky_money_{num_red_packets}_packets.csv'
    
   
    probabilities, actual_rounds = calculate_max_red_packet_probability(file_path, total_rounds, num_red_packets)
    
  
    print(f"For {num_red_packets} red packets:")
    for i, probability in enumerate(probabilities, start=1):
        print(f"The probability of the {i}th position getting the maximum red packet is: {probability:.2%}")
    

    if actual_rounds != total_rounds:
        print(f"Warning: The actual number of rounds processed was {actual_rounds}, not {total_rounds}.")


    positions = range(1, num_red_packets + 1)  
    plt.figure(figsize=(5, 3))
    plt.scatter(positions, probabilities, color='blue', label='Probability of Best Luck')  
    plt.title(f'Probability of Each Position Getting the Most Money')
    plt.xlabel('Position')
    plt.ylabel('Probability')
    plt.xticks(positions) 

    plt.legend()
    plt.savefig(f'picture/{num_red_packets}_probability_plot_packets.png') 
    plt.close() 

print('All probability plots have been saved.')

In [ ]:
from PIL import Image

images_folder = 'picture' 

output_image_filename = 'combined_image.png'
rows, cols = 5, 5
grid_positions = [(i, j) for i in range(rows) for j in range(cols)] 

test_image_path = os.path.join(images_folder, '3_probability_plot_packets.png')
test_image = Image.open(test_image_path)
width, height = test_image.size

# 创建一个新的空白图片，尺寸为5x5网格的总宽度和总高度
total_width = cols * width
total_height = rows * height
combined_image = Image.new('RGB', (total_width, total_height), (255, 255, 255))  


for position in grid_positions:
    num_packets = position[0] * cols + position[1] + 2 
    img_filename = f'{num_packets}_probability_plot_packets.png'
    img_path = os.path.join(images_folder, img_filename)
    
    if os.path.isfile(img_path):
        img = Image.open(img_path)
        combined_image.paste(img, (position[1] * width, position[0] * height))
    else:
        blank_image = Image.new('RGB', (width, height), (255, 255, 255))
        combined_image.paste(blank_image, (position[1] * width, position[0] * height))

combined_image.save(output_image_filename)
print(f'Combined image saved as {output_image_filename}')